### Import the required libraries for OpenCV and Keras

In [ ]:
from tensorflow import keras
from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np
#from config import Moodkey

### Load data and images model path 

In [2]:
# parameters for loading data and images
detection_model_path = 'files/haarcascade_frontalface_default.xml'
emotion_model_path = 'models/_mini_XCEPTION.106-0.65.hdf5'

### Load the models for face detection and trained emotion classifier

In [3]:
# hyper-parameters for bounding boxes shape
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
#EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised", "neutral"]
EMOTIONS = ["angry","disgust", "scared", "confused", "sad", "happy", "neutral"]

### Twilio package to send notifications based on mood thresholds

In [4]:
def probability_measure_func(emotion_probability, label, count):
    
    # Download the helper library from https://www.twilio.com/docs/python/install
    from twilio.rest import Client

    client = Client(account_sid, auth_token)

    if (emotion_probability>0.30 and label !="neutral"):
        print('emotion probability : ' + str(emotion_probability) + 'with label:' + str(label) + "\n")
        
        if (emotion_probability > 0.70 and label == "confused" and count ==0):
            message = client.messages \
                        .create(
                             body="Happy to answer any questions! What are you confused about?",
                             from_='+1xxxxxxxxxx',
                             to='+1xxxxxxxxxx'
                         )
            count = 1
            print(message.sid)
    
        if (emotion_probability > 0.75 and label == "confused" and count ==1):
            message = client.messages \
                        .create(
                             body="Route to chatbot for further engagement",
                             from_='+1xxxxxxxxxx',
                             to='+1xxxxxxxxxx'
                         )
            count = 1
            print(message.sid) 
            
        if (emotion_probability > 0.30 and (label == "angry" or label == "sad") and count ==1):
            message = client.messages \
                        .create(
                             body="All is well buddy! You are enough! Join Earth's mightiest heroes. Like Kevin Bacon.",
                             from_='+1xxxxxxxxxx',
                             to='+1xxxxxxxxxx'
                         )
            count = 1
            print(message.sid)
        
        if (count ==0 and emotion_probability > 0.30 and (label == "angry" or label =="sad")):
            message = client.messages \
                        .create(
                             body="Come on now! Here's a happy song mate: https://www.youtube.com/watch?v=xoqzf7x5J08",
                             from_='+1xxxxxxxxxx',
                             to='+1xxxxxxxxxx'
                         )
            count = 1
            print(message.sid)
    

### Invoke Chatbot for interaction based on emotion probability

In [5]:
def call_watson_chatbot(emotion_probability, label):
    from ibm_watson import AssistantV2
    assistant = AssistantV2(
        version='3.4.0',
        #iam_apikey= Moodkey,
        url='https://gateway.watsonplatform.net/assistant/api'
        )
    #API to call/instantiate watson chatbot here
    #I want watson chatbot to now send me a message and start interacting. Future version

### Start the video streaming and capturing student's emotions as the lecture progresses! 

In [ ]:
# starting video streaming
cv2.namedWindow('your_face')
camera = cv2.VideoCapture(0)
count = 0

    
while True:
    frame = camera.read()[1]
    #reading the frame
    frame = imutils.resize(frame,width=400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
        
    canvas = np.zeros((300, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
        # Extract the ROI of the face from the grayscale image, resize it to a fixed 48x48 pixels, and then prepare
            # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
    
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        
       # if(emotion_probability > 0.30):
        #    probability_measure_func(emotion_probability, label, count)
            
            
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5),
                (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (255, 255, 255), 2)
                cv2.putText(frameClone, label, (fX, fY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                              (0, 0, 255), 2)

    cv2.imshow('your_face', frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()